In [108]:
import numpy as np
import matplotlib.pyplot as plt

In [163]:
R = 1 # sphere radius
T = 1 # simulation time

j = 0.25 # dp
k = 0.01 # dt

p = np.arange(0, R+j, j)
t = np.arange(0, T+k, k)

n = len(p)
m = len(t)

T = np.zeros((n, m+1)) # includes ghost node

T[0, :] = 20
T[:, 0] = 100

factor = k/(j**2)

if factor > 0.5:
    print("Factor is too large")

In [164]:
for i in range(1, n):
    for j in range(1, m):
        T[i,j] = (1-2*factor)*T[i-1,j] + factor*(T[i-1,j-1] + T[i-1,j+1])
        T[i,-1] = T[i,-3]

In [160]:
# Sample data
x = np.tile(t, m)
y = np.repeat(p, n)
z = T.flatten()

# # Create a 3D figure and an axes object
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')

# # Create a 3D scatter plot
# ax.scatter(x, y, z, c='b', marker='o')

# # Set labels for each axis
# ax.set_xlabel('X-axis')
# ax.set_ylabel('Y-axis')
# ax.set_zlabel('Z-axis')

# # Set the title of the plot
# ax.set_title('3D Scatter Plot')

# # Show the plot
# plt.show()

print(len(x), len(y), len(z))

1002001 25 5010
